<a href="https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/05_TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 01 – Imports y configuración global
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.applications import DenseNet121, EfficientNetB4
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preproc
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preproc

# reproducibilidad
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# mixed precision (si GPU lo soporta)
tf.config.optimizer.set_experimental_options({'auto_mixed_precision': True})


In [2]:
!pip install -q keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/129.1 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.4 MB/s eta 0:00:00


In [2]:
import keras_tuner as kt

In [8]:
#!rm -rf /content/my_dir/hyperparam_multiclass SOLO USAR AL REINICIAR


# Estructura general

1. **Parámetros y rutas**  
2. **Función de carga de backbones**  
3. **Definición de `build_model()`**  
4. **Carga de los datasets con `tf.data`**  
5. **Instanciación y ejecución de Keras Tuner**  

---

## 1) Parámetros y rutas

- **`TRAIN_DIR`** y **`VAL_DIR`**: rutas a las carpetas de entrenamiento y validación.  
- **`IMG_SIZE`**: tamaño al que redimensionar todas las imágenes (224×224).  
- **`BATCH`**: tamaño de lote para el entrenamiento (32).  
- **`AUTOTUNE`**: para acelerar la carga de datos con prefetching automático.

---

## 2) Función `load_backbone(name, input_shape)`

Carga un **backbone pre-entrenado** de ImageNet y su función de preprocesado:

| `name`         | Modelo base           | Preprocesado                            |
|---------------|-----------------------|-----------------------------------------|
| `"densenet"`  | `DenseNet121`         | `keras.applications.densenet.preprocess_input`  |
| `"effnet_b4"` | `EfficientNetB4`      | `keras.applications.efficientnet.preprocess_input` |

- **Congelamos** (`trainable=False`) todas las capas del backbone para usarlo como extractor de características.

---

## 3) Función `build_model(hp)`

Define el **espacio de búsqueda** y construye un modelo Keras:

1. **Selección de backbone**  
   ```python
   hp.Choice('backbone', ['densenet','effnet_b4'])

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
hyperparam_multiclass.py

Búsqueda de hiperparámetros para clasificación multiclas de imágenes
(3 clases: NORMAL, PNEUMONIA-BACTERIAL, PNEUMONIA-VIRAL).

Estructura:
 1) Carga de datos con tf.data
 2) Definición de función de carga de backbones
 3) Definición de build_model() para Keras Tuner
 4) Instanciación de BayesianOptimization
 5) Ejecución de tuner.search() + análisis de resultados
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121, EfficientNetB4
import keras_tuner as kt

# Si quieres usar ViT de transformers, descomenta:
# from transformers import ViTConfig, TFViTModel

# -----------------------------------------------------------------------------
# 1) Parámetros y rutas
# -----------------------------------------------------------------------------
TRAIN_DIR = "/content/drive/MyDrive/Capstone/data/multiclass/train"
VAL_DIR   = "/content/drive/MyDrive/Capstone/data/multiclass/val"

IMG_SIZE = (224, 224)
BATCH    = 32
AUTOTUNE = tf.data.AUTOTUNE

# -----------------------------------------------------------------------------
# 2) Función para cargar distintos backbones + preprocesado
# -----------------------------------------------------------------------------
def load_backbone(name, input_shape=(*IMG_SIZE, 3)):
    """
    name: 'densenet', 'effnet_b4' o 'vit_b16'
    devuelve: (modelo_base_sin_top, función_preprocesado)
    """
    if name == 'densenet':
        base = DenseNet121(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
        preproc = keras.applications.densenet.preprocess_input

    elif name == 'effnet_b4':
        base = EfficientNetB4(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
        preproc = keras.applications.efficientnet.preprocess_input

    # elif name == 'vit_b16':
    #     # Ejemplo con HuggingFace Vision Transformer
    #     config = ViTConfig(
    #         image_size=224,
    #         num_labels=0,
    #         hidden_size=768,
    #         num_hidden_layers=12,
    #         num_attention_heads=12,
    #         patch_size=16
    #     )
    #     base = TFViTModel.from_pretrained(
    #         'google/vit-base-patch16-224-in21k',
    #         config=config
    #     )
    #     preproc = lambda x: tf.cast(x, tf.float32) / 255.0

    else:
        raise ValueError(f"Backbone desconocido: {name}")

    base.trainable = False
    return base, preproc

# -----------------------------------------------------------------------------
# 3) build_model() para Keras Tuner
# -----------------------------------------------------------------------------
def build_model(hp):
    # 3.1) Elige backbone
    backbone_name = hp.Choice('backbone', ['densenet', 'effnet_b4'])
    base, preproc_fn = load_backbone(backbone_name)

    # 3.2) Input + preprocesado + backbone
    inputs = keras.Input(shape=(*IMG_SIZE,3), name='input_image')
    x = preproc_fn(inputs)
    x = base(x, training=False)

    # 3.3) Pooling global + Dropout (hiperparámetro)
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dropout(
        hp.Float('dropout', 0.0, 0.5, step=0.1),
        name='dropout'
    )(x)

    # 3.4) Capa de salida multiclas (3 neuronas + softmax)
    outputs = layers.Dense(3, activation='softmax', name='predictions')(x)

    model = keras.Model(inputs, outputs, name='multiclass_model')

    # 3.5) Compilar con loss y métricas adecuadas
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('lr', 1e-5, 1e-3, sampling='log'),
            name='adam_lr'
        ),
        loss='categorical_crossentropy',
        metrics=[
            keras.metrics.CategoricalAccuracy(name='acc'),
            # Si quieres AUC multiclas:
            # keras.metrics.AUC(name='auc', multi_label=True, num_labels=3)
        ]
    )

    return model

# -----------------------------------------------------------------------------
# 4) Carga de datasets con tf.data
# -----------------------------------------------------------------------------
def get_datasets():
    train_ds = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        labels='inferred',
        label_mode='categorical',
        image_size=IMG_SIZE,
        batch_size=BATCH,
        shuffle=True,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        VAL_DIR,
        labels='inferred',
        label_mode='categorical',
        image_size=IMG_SIZE,
        batch_size=BATCH,
        shuffle=False,
    )
    # Prefetch para rendimiento
    train_ds = train_ds.prefetch(AUTOTUNE)
    val_ds   = val_ds.prefetch(AUTOTUNE)
    return train_ds, val_ds

# -----------------------------------------------------------------------------
# 5) Instanciar y ejecutar Keras Tuner
# -----------------------------------------------------------------------------
def run_tuner():
    train_ds, val_ds = get_datasets()

    tuner = kt.BayesianOptimization(
        build_model,
        objective='val_acc',      # usamos accuracy; si cambias a AUC, pon 'val_auc'
        max_trials=20,
        directory='my_dir',
        project_name='hyperparam_multiclass'
    )

    # Early stopping sobre la métrica de validación
    stop_early = tf.keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=3,
        mode='max',
        restore_best_weights=True
    )

    tuner.search(
        train_ds,
        validation_data=val_ds,
        epochs=10,
        callbacks=[stop_early]
    )

    # 6) Resumen de resultados
    tuner.results_summary()
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    print("\n>>> Mejores hiperparámetros encontrados:")
    for param, val in best_hp.values.items():
        print(f"    {param}: {val}")

#run_tuner()




Trial 20 Complete [00h 02m 37s]
val_acc: 0.5454545617103577

Best val_acc So Far: 0.5909090638160706
Total elapsed time: 01h 55m 18s
Results summary
Results in my_dir/hyperparam_multiclass
Showing 10 best trials
Objective(name="val_acc", direction="max")

Trial 10 summary
Hyperparameters:
backbone: densenet
dropout: 0.30000000000000004
lr: 0.0006977759410136506
Score: 0.5909090638160706

Trial 07 summary
Hyperparameters:
backbone: densenet
dropout: 0.0
lr: 1.3348402960430868e-05
Score: 0.5909090638160706

Trial 01 summary
Hyperparameters:
backbone: densenet
dropout: 0.4
lr: 0.00010200445184344612
Score: 0.5909090638160706

Trial 12 summary
Hyperparameters:
backbone: densenet
dropout: 0.0
lr: 2.4026541531091054e-05
Score: 0.5909090638160706

Trial 16 summary
Hyperparameters:
backbone: densenet
dropout: 0.1
lr: 8.47434662661372e-05
Score: 0.5909090638160706

Trial 17 summary
Hyperparameters:
backbone: densenet
dropout: 0.4
lr: 7.191260845067741e-05
Score: 0.5909090638160706

Trial 03 sum

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
def configure_gpu():
    # Permite que TF crezca la memoria en lugar de reservarla toda de golpe
    gpus = tf.config.list_physical_devices('GPU')
    if not gpus:
        print("⚠️  No GPUs detectadas. Saliendo.")
        exit(1)
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPUs disponibles:", gpus)

print("=== Test GPU con TensorFlow ===")
configure_gpu()
tf.debugging.set_log_device_placement(False)

=== Test GPU con TensorFlow ===
GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-07-01 01:23:36.125315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-01 01:23:36.129381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-01 01:23:36.129803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [8]:
rm -rf  my_dir/chexnet_hyperparam_refined/tuner0.json

In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
hyperparam_multiclass_with_chexnet_refined.py

Versión refinada para usar tu CheXNet SavedModel legacy en Keras 3,
con toggle de rutas local vs Google Drive, data augmentation,
normalización tras CheXNet (sin pooling) y búsqueda de hiperparámetros centrada.
"""

import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

tf.debugging.set_log_device_placement(False)

# -----------------------------------------------------------------------------
# SWITCH de entorno: True para Drive, False para local (cwd)
# -----------------------------------------------------------------------------
USE_DRIVE = False
if USE_DRIVE:
    BASE_PATH = "/content/drive/MyDrive/Capstone"
else:
    BASE_PATH = "/workspace"

# -----------------------------------------------------------------------------
# 0) Directorio de tu SavedModel de CheXNet
# -----------------------------------------------------------------------------
CHEXNET_SM_DIR = os.path.join(BASE_PATH, "saved_models", "chexnet")
if not os.path.isdir(CHEXNET_SM_DIR):
    raise FileNotFoundError(f"No existe el SavedModel en {CHEXNET_SM_DIR}")

# Asegurar carpeta variables/
vars_folder = os.path.join(CHEXNET_SM_DIR, "variables")
if not os.path.isdir(vars_folder):
    os.makedirs(vars_folder, exist_ok=True)
    for fn in os.listdir(CHEXNET_SM_DIR):
        if fn.startswith("variables.") and os.path.isfile(os.path.join(CHEXNET_SM_DIR, fn)):
            shutil.move(
                os.path.join(CHEXNET_SM_DIR, fn),
                os.path.join(vars_folder, fn)
            )

# -----------------------------------------------------------------------------
# 1) Cargo el módulo SavedModel y extraigo la función de inferencia
# -----------------------------------------------------------------------------
chexnet_module = tf.saved_model.load(CHEXNET_SM_DIR)
infer_fn        = chexnet_module.signatures["serving_default"]
out_key         = list(infer_fn.structured_outputs.keys())[0]

# -----------------------------------------------------------------------------
# 2) Parámetros y rutas de datos
# -----------------------------------------------------------------------------
TRAIN_DIR = os.path.join(BASE_PATH, "multiclass", "train")
VAL_DIR   = os.path.join(BASE_PATH, "multiclass", "val")
IMG_SIZE  = (224, 224)
BATCH     = 32
AUTOTUNE  = tf.data.AUTOTUNE

# -----------------------------------------------------------------------------
# 3) Creación de datasets 
# -----------------------------------------------------------------------------
def get_datasets(batch_size):
    augmentation = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.05),
        layers.RandomBrightness(factor=0.1),
    ], name="augmentation")

    def preprocess_train(images, labels):
        images = augmentation(images)
        images = keras.applications.densenet.preprocess_input(images)
        return images, labels

    def preprocess_val(images, labels):
        images = keras.applications.densenet.preprocess_input(images)
        return images, labels

    train_ds = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        labels="inferred",
        label_mode="categorical",
        image_size=IMG_SIZE,
        batch_size=batch_size,
        shuffle=True,
    ).map(preprocess_train, num_parallel_calls=AUTOTUNE)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        VAL_DIR,
        labels="inferred",
        label_mode="categorical",
        image_size=IMG_SIZE,
        batch_size=batch_size,
        shuffle=False,
    ).map(preprocess_val, num_parallel_calls=AUTOTUNE)

    return train_ds.prefetch(AUTOTUNE), val_ds.prefetch(AUTOTUNE)

# -----------------------------------------------------------------------------
# 4) build_model() para Keras Tuner con normalización (sin pooling)
# -----------------------------------------------------------------------------
def build_model(hp):
    inputs = keras.Input(shape=(*IMG_SIZE, 3), name="input_image")
    x = keras.applications.densenet.preprocess_input(inputs)

    # Llamada al bloque CheXNet
    def call_chexnet(tensor):
        return infer_fn(tensor)[out_key]
    x = layers.Lambda(call_chexnet, name="chexnet_block")(x)

    # Normalización de la salida 2D (batch, 14)
    x = layers.BatchNormalization(name="bn_chexnet")(x)

    # Dropout ajustable sobre ese vector de 14 features
    x = layers.Dropout(
        hp.Float("dropout", min_value=0.0, max_value=0.2, step=0.05),
        name="dropout"
    )(x)

    outputs = layers.Dense(3, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs, outputs, name="chexnet_multiclass")

    # Optimizer y learning rate
    lr = hp.Float("learning_rate", 1e-5, 1e-3, sampling="log")
    opt_choice = hp.Choice("optimizer", ["adam", "adamw"])
    if opt_choice == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    else:
        optimizer = keras.optimizers.experimental.AdamW(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")]
    )
    return model

# -----------------------------------------------------------------------------
# 5) Ejecución del tuner con callbacks mejorados
# -----------------------------------------------------------------------------
def run_tuner():
    train_ds, val_ds = get_datasets(batch_size=BATCH)

    tuner = kt.BayesianOptimization(
        build_model,
        objective="val_acc",
        max_trials=20,
        directory="my_dir",
        project_name="chexnet_hyperparam_refined"
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_acc", patience=5, mode="max", restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6
        )
    ]

    tuner.search(
        train_ds,
        validation_data=val_ds,
        epochs=20,
        callbacks=callbacks
    )

    tuner.results_summary()
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    print("\n>>> Mejores hiperparámetros refinados con CheXNet:")
    for param, val in best_hp.values.items():
        print(f"    {param}: {val}")

if __name__ == "__main__":
    run_tuner()


Trial 20 Complete [00h 01m 05s]
val_acc: 0.3181818127632141

Best val_acc So Far: 0.5
Total elapsed time: 00h 26m 39s
Results summary
Results in my_dir/chexnet_hyperparam_refined
Showing 10 best trials
Objective(name="val_acc", direction="max")

Trial 09 summary
Hyperparameters:
dropout: 0.0
learning_rate: 1.0779892690656582e-05
optimizer: adamw
Score: 0.5

Trial 16 summary
Hyperparameters:
dropout: 0.0
learning_rate: 1.2100818369205357e-05
optimizer: adamw
Score: 0.5

Trial 17 summary
Hyperparameters:
dropout: 0.0
learning_rate: 1.1532423223506732e-05
optimizer: adamw
Score: 0.5

Trial 04 summary
Hyperparameters:
dropout: 0.0
learning_rate: 0.00045454167020149807
optimizer: adam
Score: 0.4545454680919647

Trial 11 summary
Hyperparameters:
dropout: 0.2
learning_rate: 0.0005728965578955454
optimizer: adamw
Score: 0.4545454680919647

Trial 15 summary
Hyperparameters:
dropout: 0.0
learning_rate: 1e-05
optimizer: adamw
Score: 0.4545454680919647

Trial 02 summary
Hyperparameters:
dropout: 0

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
hyperparam_multiclass_with_chexnet_final.py

Versión estable:
 - Toggle de rutas local vs Drive
 - Data augmentation suave
 - Normalización tras CheXNet
 - Head con capa densa intermedia + Dropout
 - Búsqueda de hiperparámetros: lr, optimizer, dropout, dense_units
 - Callbacks: EarlyStopping + ReduceLROnPlateau
"""

import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

tf.debugging.set_log_device_placement(False)

# -----------------------------------------------------------------------------
# SWITCH de entorno: True para Drive, False para local
# -----------------------------------------------------------------------------
USE_DRIVE = False
if USE_DRIVE:
    BASE_PATH = "/content/drive/MyDrive/Capstone"
else:
    BASE_PATH = "/workspace"

# -----------------------------------------------------------------------------
# (Opcional) Limpia resultados previos del tuner
# -----------------------------------------------------------------------------
RESULTS_DIR = os.path.join(BASE_PATH, "my_dir", "chexnet_hyperparam_final")
if os.path.isdir(RESULTS_DIR):
    shutil.rmtree(RESULTS_DIR)

# -----------------------------------------------------------------------------
# 0) Carga de tu SavedModel de CheXNet y firma de inferencia
# -----------------------------------------------------------------------------
CHEXNET_SM_DIR = os.path.join(BASE_PATH, "saved_models", "chexnet")
if not os.path.isdir(CHEXNET_SM_DIR):
    raise FileNotFoundError(f"No existe el SavedModel en {CHEXNET_SM_DIR}")

chexnet_module = tf.saved_model.load(CHEXNET_SM_DIR)
infer_fn        = chexnet_module.signatures["serving_default"]
out_key         = list(infer_fn.structured_outputs.keys())[0]

# -----------------------------------------------------------------------------
# 1) Parámetros y rutas de datos
# -----------------------------------------------------------------------------
TRAIN_DIR = os.path.join(BASE_PATH, "multiclass", "train")
VAL_DIR   = os.path.join(BASE_PATH, "multiclass", "val")
IMG_SIZE  = (224, 224)
BATCH     = 32
AUTOTUNE  = tf.data.AUTOTUNE

# -----------------------------------------------------------------------------
# 2) Data augmentation suave y pipeline tf.data
# -----------------------------------------------------------------------------
def get_datasets(batch_size):
    aug = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomZoom(0.02),
        layers.RandomBrightness(factor=0.05),
    ], name="augmentation")

    def preprocess_train(images, labels):
        images = aug(images)
        images = keras.applications.densenet.preprocess_input(images)
        return images, labels

    def preprocess_val(images, labels):
        images = keras.applications.densenet.preprocess_input(images)
        return images, labels

    train_ds = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR, labels="inferred", label_mode="categorical",
        image_size=IMG_SIZE, batch_size=batch_size, shuffle=True
    ).map(preprocess_train, num_parallel_calls=AUTOTUNE)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        VAL_DIR, labels="inferred", label_mode="categorical",
        image_size=IMG_SIZE, batch_size=batch_size, shuffle=False
    ).map(preprocess_val, num_parallel_calls=AUTOTUNE)

    return train_ds.prefetch(AUTOTUNE), val_ds.prefetch(AUTOTUNE)

# -----------------------------------------------------------------------------
# 3) Definición del modelo para Keras Tuner
# -----------------------------------------------------------------------------
def build_model(hp):
    inputs = keras.Input(shape=(*IMG_SIZE, 3), name="input_image")
    x = keras.applications.densenet.preprocess_input(inputs)

    # Invoca al SavedModel CheXNet
    def call_chexnet(t):
        return infer_fn(t)[out_key]
    x = layers.Lambda(call_chexnet, name="chexnet_block")(x)

    # Normalización de la salida (batch, 14)
    x = layers.BatchNormalization(name="bn_chexnet")(x)

    # Capa oculta intermedia
    x = layers.Dense(
        units=hp.Choice("dense_units", [64, 128]),
        activation="relu",
        name="hidden_dense"
    )(x)
    x = layers.Dropout(
        rate=hp.Float("dropout", 0.0, 0.2, step=0.05),
        name="dropout"
    )(x)

    outputs = layers.Dense(3, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs, outputs, name="chexnet_multiclass_final")

    # Optimizer y learning rate
    lr = hp.Float("learning_rate", 1e-6, 1e-4, sampling="log")
    opt = hp.Choice("optimizer", ["adamw", "adam"])
    if opt == "adamw":
        optimizer = keras.optimizers.experimental.AdamW(learning_rate=lr)
    else:
        optimizer = keras.optimizers.Adam(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")]
    )
    return model

# -----------------------------------------------------------------------------
# 4) Ejecutar el tuner con callbacks avanzados
# -----------------------------------------------------------------------------
def run_tuner():
    train_ds, val_ds = get_datasets(batch_size=BATCH)

    tuner = kt.BayesianOptimization(
        hypermodel=build_model,
        objective="val_acc",
        max_trials=30,
        directory=os.path.join(BASE_PATH, "my_dir"),
        project_name="chexnet_hyperparam_final"
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_acc", patience=7, mode="max", restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=4, min_lr=1e-7
        )
    ]

    tuner.search(
        train_ds,
        validation_data=val_ds,
        epochs=25,
        callbacks=callbacks
    )

    tuner.results_summary()
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    print("\n>>> Mejores hiperparámetros finales:")
    for param, val in best_hp.values.items():
        print(f"    {param}: {val}")

if __name__ == "__main__":
    run_tuner()


Trial 30 Complete [00h 02m 24s]
val_acc: 0.3636363744735718

Best val_acc So Far: 0.5
Total elapsed time: 00h 58m 57s
Results summary
Results in /workspace/my_dir/chexnet_hyperparam_final
Showing 10 best trials
Objective(name="val_acc", direction="max")

Trial 12 summary
Hyperparameters:
dense_units: 128
dropout: 0.1
learning_rate: 2.8507137885985768e-05
optimizer: adam
Score: 0.5

Trial 00 summary
Hyperparameters:
dense_units: 128
dropout: 0.1
learning_rate: 3.381099566791077e-05
optimizer: adam
Score: 0.4545454680919647

Trial 06 summary
Hyperparameters:
dense_units: 128
dropout: 0.0
learning_rate: 2.117018483773315e-06
optimizer: adamw
Score: 0.4545454680919647

Trial 14 summary
Hyperparameters:
dense_units: 128
dropout: 0.1
learning_rate: 2.975684481471653e-05
optimizer: adam
Score: 0.4545454680919647

Trial 15 summary
Hyperparameters:
dense_units: 128
dropout: 0.1
learning_rate: 6.729078586838823e-05
optimizer: adam
Score: 0.4545454680919647

Trial 21 summary
Hyperparameters:
dens